<h1 >Steps for Analysing Tweets with NLP and additional Tools</h1>

<bold>Data Visualization</bold>
    1. Chloropeth Map of our Dataset with the help of Tableau
    2. Length of Tweets of Train and Test
    3. Which Keyword is related to disaster?
    4. Wordcloud of Tweets
    5. Ngrams of Most frequently occuring words
<bold>Data Preprocessing</bold>
    1. Cleaning with Regex
    2. Converting the text to TFIDF
<bold>Train and Test Model</bold>
    1. Use the Naive Bayes Algorithmn
    2. Save the output

# **Additionally**
<bold>Using Sentiment analysizing,Topic Modelling</bold>
    To check dataset is not bias to any side

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
train.shape

In [ ]:
test= pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test.shape

In [ ]:
train.head(5)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

**Keywords used in Tweets**

In [ ]:
train['keyword'].unique()

In [ ]:
train['location'].unique()

# Data Visualisations

In [ ]:
# Import Tableau Visualisation 
from IPython.display import IFrame
IFrame('https://public.tableau.com/views/DisasterTweets_16128035695960/Sheet1?:language=en&:retry=yes&:display_count=y&:origin=viz_share_link', width=1000, height=925)

In [ ]:
train[train['keyword'] == 'accident'].head(5)


#### Length of Train Tweets Vs Test Tweet

In [ ]:
tweetLengthTrain = train['text'].str.len()
tweetLengthTest = test['text'].str.len()

plt.hist(tweetLengthTrain,bins=20,label='Train_Tweet')
plt.hist(tweetLengthTest,bins=20,label='Test_Tweet')
plt.legend()
plt.show()

In [ ]:
train["keyword"].fillna("no_keywords",inplace = True)
test["keyword"].fillna("no_keywords",inplace = True)
train["location"].fillna("no_location",inplace=True)
test["location"].fillna("no_location",inplace=True)

Which Keyword is Related to a Disaster?

In [ ]:
train["target_mean"] = train.groupby("keyword")["target"].transform("mean")

plt.figure(figsize=(8,72))

sns.countplot(y=train.sort_values("target",ascending=False)["keyword"],hue=\
              train.sort_values("target",ascending=False)["target"])
plt.tick_params(axis="x",labelsize=15)
plt.tick_params(axis="y",labelsize=15)
plt.title("Target Distribution in Keywords")
plt.show()

In [ ]:
train_df=train
test_df=test

In [ ]:
combine = train.append(test,ignore_index=True)
print('Shape of new Dataset:',combine.shape)
combine.tail()

In [ ]:
text_tweet = combine["text"]
text_tweet.tail()

In [ ]:
df =  pd.DataFrame(combine[['text']])
df.head(7)

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation]) #all punctuations 
    text = re.sub('[0-9]+', '', text) #all numbers
    return text

df['Tweet_punct'] = df['text'].apply(lambda x: remove_punct(x))
df.head(10)

In [ ]:
df["Tweet_punct"].drop_duplicates(inplace = True)
df["Tweet_punct"].shape

In [ ]:
combine["Tweet_punct"] = df['Tweet_punct']

In [ ]:
combine.head()

In [ ]:
#important libraries for preprocessing using NLTK
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import * 

In [ ]:
df = combine[combine['Tweet_punct'].notna()]
df.head()

In [ ]:
#Tokenize the tweets

def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tweet_tokenized'] = df['Tweet_punct'].apply(lambda x: tokenization(x.lower()))
df.head(10)

# Stemming the Tokenized Tweets

**Stemming algorithms work by cutting off the end or the beginning of the word, taking into account a list of common prefixes and suffixes that can be found in an inflected word. This indiscriminate cutting can be successful in some occasions, but not always, and that is why we affirm that this approach presents some limitations.**
[Further Content](http://https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)

In [ ]:
stemmer = PorterStemmer()
df['Tweet_tokenized'] = df['Tweet_tokenized'].apply(lambda x : [stemmer.stem(i) for i in x]  )

In [ ]:
df.head()

In [ ]:
#Joining the tokenized tweets

for i in range(len(df['Tweet_tokenized'])):
    df['Tweet_tokenized'][i] = ' '.join(df['Tweet_tokenized'][i])    
df['cleanedText'] = df['Tweet_tokenized']

In [ ]:
df.head()

In [ ]:
# Creating word Cloud for all Words in all tweets
from wordcloud import WordCloud
allWords = ' '.join([text for text in df['cleanedText']])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(allWords)
plt.figure(figsize=(10, 10)) 
plt.imshow(wordcloud, interpolation="bilinear") 
plt.axis('off') 
plt.show()

# **Sentiment Analysis**
Sentiment analysis is contextual mining of text which identifies and extracts subjective information in source material, and helping a business to understand the social sentiment of their brand, product or service while monitoring online conversations. 
[Sentiment Analysis Further Content](http://https://towardsdatascience.com/sentiment-analysis-concept-analysis-and-applications-6c94d6f58c17)

In [ ]:
from textblob import TextBlob

def sentiment_polarity(text):
    tweet_text = TextBlob(str(text))
    sentiment_value = tweet_text.sentiment.polarity
    return sentiment_value

df['Tweet_Polarity'] = df['cleanedText'].apply(lambda x: sentiment_polarity(x.lower()))
df.head(10)

In [ ]:
def sentiment_analysis(value):
    sentiment=""
    if(value<0.0):
        sentiment = "negative"
    elif(value>0.0):
        sentiment = "positive"
    else:
        sentiment = "neutral"
    return sentiment

df['Tweet_Sentiments'] = df['Tweet_Polarity'].apply(lambda x: sentiment_analysis(float(x)))
df[["Tweet_punct","Tweet_Polarity"]].head(10)

In [ ]:
import seaborn as sns
ax = sns.countplot(x="Tweet_Sentiments", data=df)

This is done to check if the overall distribution of data was neutral. It is important as we are checking which tweet is a disaster so we have to a dataset which is not bias towards either side.

# **TF-IDF**

Term Frequency, which measures how frequently a term occurs in a document. Inverse Document Frequency, which measures how important a term is with respect to the document. TFIDF gives us a array which shows how frequeeently important occurs while taking into consideration conditions like Long Tail.Which can be managed by taking Log of the matrix.
[Further content](http://https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(df['cleanedText']).toarray()
X

In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tweet_tokenized'] = df['cleanedText'].apply(lambda x: tokenization(x.lower()))
df.head(10)

# **Topic Modeling-LDA**
By Topic Modelling we can check that which words where said highest and make sense of it by finding few common topics by the means ofprobabality

In [ ]:
from gensim.corpora import Dictionary

#create dictionary
text_dict = Dictionary(df.Tweet_tokenized)

#view integer mappings
text_dict.token2id

In [ ]:
tweets_bow = [text_dict.doc2bow(tweet) for tweet in df['Tweet_tokenized']]
tweets_bow

In [ ]:
from gensim.models.ldamodel import LdaModel

k = 10
tweets_lda = LdaModel(tweets_bow,
                      num_topics = k,
                      id2word = text_dict,
                      random_state = 1,
                      passes=10)

tweets_lda.show_topics()

# **Text Grouping by Regex**
A RegEx, or Regular Expression, is a sequence of characters that forms a search pattern.
RegEx can be used to check if a string contains the specified search pattern.
[Further Content](http://https://www.w3schools.com/python/python_regex.asp)

In [ ]:
# word_count
train_df["word_count"] = train_df["text"].map(lambda x: len(str(x).split()))
test_df["word_count"] = test_df["text"].map(lambda x: len(str(x).split()))

# unique_word_count 
train_df["unique_word_count"] = train_df["text"].map(lambda x:len(set(str(x).split())))
test_df["unique_word_count"] = test_df["text"].map(lambda x:len(set(str(x).split())))

# stop_word_count
train_df["stop_word_count"] = train_df["text"].map(lambda x: len([elt for elt in str(x).lower().split() \
                                                                  if elt in stopwords.words("english")]))
test_df["stop_word_count"] = test_df["text"].map(lambda x:len([elt for elt in str(x).lower().split()\
                                                              if elt in stopwords.words("english")]))
# url_count 
train_df["url_count"] = train_df["text"].map(lambda x : len([w for w in str(x).lower().split()\
                                                            if 'http' or 'https' in w]))
test_df["url_count"] = test_df["text"].map(lambda x : len([w for w in str(x).lower().split()\
                                                          if "http" or "https" in w ]))
# mean_word_length
train_df["mean_word_length"] = train_df["text"].map(lambda x : np.mean([len(word) for word in x.split()]))
test_df["mean_word_length"] = test_df["text"].map(lambda x : np.mean([len(word) for word in x.split()]))

#char_count 
train_df["char_count"] = train_df["text"].map(lambda x:len(str(x)))
test_df["char_count"] = test_df["text"].map(lambda x:len(str(x)))

#punctuation_count
train_df["punctuation_count"] = train_df["text"].map(lambda x: len([elt for elt in str(x) if elt in string.punctuation]))
test_df["punctuation_count"] = test_df["text"].map(lambda x:len([elt for elt in str(x) if elt in string.punctuation]))
#hashtag_count
train_df["hashtag_count"] = train_df["text"].apply(lambda x:len([c for c in str(x) if c=="#"]))
test_df["hashtag_count"] = test_df["text"].apply(lambda x:len([c for c in str(x) if c=="#"]))

#mention_count
train_df["mention_count"] = train_df["text"].map(lambda x: len([c for c in str(x) if c=="@"]))
test_df["mention_count"] = test_df["text"].map(lambda x: len([c for c in str(x) if c=="@"]))

**Plotting various Key Features**

In [ ]:
# Heureunder, we will explore the distribution of each mega-feature per target and per dataset
# (train & test)
META_FEATURES = ["word_count","unique_word_count","stop_word_count","url_count",\
                 "mean_word_length","char_count","punctuation_count","hashtag_count","mention_count"]
fig,ax = plt.subplots(nrows=len(META_FEATURES),ncols=2,figsize=(20,50),dpi=100)
mask = train_df["target"]==1
for i,feature in enumerate(META_FEATURES):

    
   sns.distplot(train_df[mask][feature],ax=ax[i,0],label="Disaster",kde=False)
   sns.distplot(train_df[~mask][feature],ax=ax[i,0],label="Not Disaster",kde=False)
   ax[i,0].set_title("{} target distribution in trainning dataset".format(feature),fontsize=13)
 
   sns.distplot(train_df[feature],ax=ax[i,1],label="Train Dataset",kde=False)
   sns.distplot(test_df[feature],ax=ax[i,1],label="Test Dataset",kde=False)
   ax[i,1].set_title("{} training and test dataset distributions ".format(feature),fontsize=13)
   for j in range(2):
        ax[i,j].set_xlabel(" ")
        ax[i,j].tick_params(axis="x",labelsize=13)
        ax[i,j].tick_params(axis="y",labelsize=13)
        ax[i,j].legend()
plt.show()

# **Checking for Data imbalance**
It is important factor for determining whether we can mine data that can gives us answers under all conditions. In this case it is a balanced dataset[Further Content](http://https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(20,6))

train_df.groupby("target").count()["id"].plot(kind="pie",labels=["Not Disaster","Disaster"],\
                                              autopct="%1.1f pourcents",ax=ax[0])
sns.countplot(x=train_df["target"],hue=train_df["target"],ax=ax[1])
ax[1].set_xticklabels(["Non Disaster","Disaster"])
ax[0].tick_params(axis="x",labelsize=15)
ax[0].tick_params(axis="y",labelsize=15)
ax[0].set_ylabel("")
ax[1].tick_params(axis="x",labelsize=15)
ax[1].tick_params(axis="y",labelsize=15)
ax[0].set_title("Target distribution in training set",fontsize=13)
ax[1].set_title("Target count in training set",fontsize=13)

# **Creating high frequent grouped words in form of Ngrams**
Simply put, an n-gram is a sequence of n words where n is a discrete number that can range from 1 to infinity!
[Further Content](http://https://towardsdatascience.com/from-dataframe-to-n-grams-e34e29df3460)

In [ ]:
# create the function which will be able to generate n_grams for each row of dataset.
def gen_n_grams(text,n_grams=1):
    """ This function allow to extract the n_gram in the introduced text.
    
      @param text(str): the text that the function, will use to extract features (n_grams).
      @param n_grams(int): the length of n_gram, that we will use.
      @return ngrams(list): list of the ngrams in the intriduced text.
    """
    tokens = [token for token in str(text).lower().split() if token not in stopwords.words("english")]
    ngrams = zip(*[tokens[i:] for i in range(n_grams)])
    
    return [" ".join(gram) for gram in ngrams]

# create the function which will be able to generate dataframe of n_gram features for disaster
# and non disaster tweets.
def gen_df_ngrams(n_grams=1):
    """ This function, allow to generate dataframes for n_grams in disaster tweets and non 
        disaster tweet
    """
    mask = train_df["target"]==1
    disaster_unigrams = defaultdict(int)
    non_disaster_unigrams = defaultdict(int)
    
    for tweet in train_df.loc[mask,"text"].values:
        for gram in gen_n_grams(tweet,n_grams=n_grams):
            disaster_unigrams[gram] +=1
    for tweet in train_df.loc[~mask,"text"].values:
        for gram in gen_n_grams(tweet,n_grams=n_grams):
            non_disaster_unigrams[gram] +=1
    df_disaster_n_grams = pd.DataFrame(sorted(disaster_unigrams.items(),reverse=True,key=\
                                              lambda item:item[1]))
    df_non_disaster_n_grams = pd.DataFrame(sorted(non_disaster_unigrams.items(),reverse=True,key=\
                                                  lambda item:item[1]))
    return df_disaster_n_grams,df_non_disaster_n_grams

# Define function, which allow to plot the N most occured n_gram in disaster tweet 
# and non disaster tweet.

def plot_ngrams(df_disaster_n_grams,df_non_disaster_unigrams,N=100,n_grams=1):
    """This function,allow to plot the top most n_grams in disaster tweet and non disaser tweet.
    """
    fig,ax = plt.subplots(1,2,figsize=(18,50))
    sns.barplot(y=df_disaster_n_grams[0].values[:N],x=df_disaster_n_grams[1].values[:N],ax=ax[0],\
               color="red")
    for i in range(2):
        ax[i].tick_params(axis="x",labelsize=15)
        ax[i].tick_params(axis="y",labelsize=15)
        ax[i].set_xlabel("Occurences")
        ax[i].spines["right"].set_visible(False)
    sns.barplot(y=df_non_disaster_unigrams[0].values[:N],x=df_non_disaster_unigrams[1].values[:N],\
               ax=ax[1],color="green")
    ax[0].set_title("Top most {} {}_grams for disaster tweets".format(N,n_grams),size=15)
    ax[1].set_title("Top most {} {}_grams for non disaster tweets".format(N,n_grams),size=15)

In [ ]:
df_disaster_unigrams,df_non_disaster_unigrams = gen_df_ngrams()
plot_ngrams(df_disaster_unigrams,df_non_disaster_unigrams)

In [ ]:
df_disaster_unigrams,df_non_disaster_unigrams = gen_df_ngrams(n_grams=2)
plot_ngrams(df_disaster_unigrams,df_non_disaster_unigrams,n_grams=2)

In [ ]:
# extract the most 100 bigrams per target (Disaster and not Disaster) when n_grams=3
df_disaster_unigrams,df_non_disaster_unigrams = gen_df_ngrams(n_grams=3)
plot_ngrams(df_disaster_unigrams,df_non_disaster_unigrams,n_grams=3)

# **Data preprocessing for Modeling**

In [ ]:
def process_text(text):
    """
    Removes punctuations(if any), stopwords and returns a list words
    """
    rm_pun = [char for char in text if char not in string.punctuation]
    rm_pun = ''.join(rm_pun)
    
    return [word for word in rm_pun.split() if word.lower() not in stopwords.words('english')]

In [ ]:
cv = CountVectorizer(analyzer=process_text).fit(train_df['text'])

In [ ]:
train_data=train_df
test_data=test_df

In [ ]:
print(len(cv.vocabulary_))

**Sample Data for TFIDF**

In [ ]:
text10 = train_data['text'][9]
text10

In [ ]:
cv10 = cv.transform([text10])
print(cv10)

In [ ]:
print(cv.get_feature_names()[5375])
print(cv.get_feature_names()[11579])
print(cv.get_feature_names()[11856])
print(cv.get_feature_names()[13190])
print(cv.get_feature_names()[25334])

In [ ]:
cv10.shape

In [ ]:
train_data_cv = cv.transform(train_data['text'])
print(train_data_cv.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer().fit(train_data_cv)

In [ ]:
tfidf10 = tfidf.transform(cv10)
print(tfidf10)

In [ ]:
train_data_tfidf = tfidf.transform(train_data_cv)
print(train_data_tfidf.shape)

# **Naive Bayes Classifier** 
Under the assumption that each data point is independent.It is a probalistic modelling Techinique in which probability of a tuple of a condition (Y|x) is the maximum of the two products of all the independent variables of either Yes or no condition.[Further Reading](http://https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
target_model = MultinomialNB().fit(train_data_tfidf, train_data['target'])

In [ ]:
print('predicted:', target_model.predict(tfidf10)[0])
print('expected:', train_data.target[9])

# **Predictions for test data**


In [ ]:
test_data_cv = cv.transform(test_data['text'])

test_data_tfidf = tfidf.transform(test_data_cv)

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
sample_submission['target'] = target_model.predict(test_data_tfidf)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.to_csv('/kaggle/working/submission.csv', index=False)